In [0]:
import os
import re
import csv
import gensim 
import codecs
import warnings 
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation

from nltk.tokenize import sent_tokenize, word_tokenize 
from gensim.models import KeyedVectors , Word2Vec
from keras.preprocessing import text
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
import sys

# Preparing Data

In [0]:
docs=[]
directory = os.fsencode('20news-bydate-train') # Train Data File with 90% Data
output_train=[]
d = [0]*20
p = -1
j = os.listdir(directory)
j.sort()
for file in j:
    print(file)
    p+=1
    d[p]=1
    for first in os.listdir(directory+b'/'+file):
        k = directory + b'/' + file + b'/' + first
        with open(k.decode('utf-8'),'r',errors='ignore') as file2:
            output_train.append(d.copy())
            docs.append(file2.read())
    d[p]=0

# Creates a list of docs. Each item is a list of all words of the doc.
m=[]
for i in docs:
    i = i.replace('\n',' ')
    m.append(text.text_to_word_sequence(i))
train_data1 = m

b'alt.atheism'
b'comp.graphics'
b'comp.os.ms-windows.misc'
b'comp.sys.ibm.pc.hardware'
b'comp.sys.mac.hardware'
b'comp.windows.x'
b'misc.forsale'
b'rec.autos'
b'rec.motorcycles'
b'rec.sport.baseball'
b'rec.sport.hockey'
b'sci.crypt'
b'sci.electronics'
b'sci.med'
b'sci.space'
b'soc.religion.christian'
b'talk.politics.guns'
b'talk.politics.mideast'
b'talk.politics.misc'
b'talk.religion.misc'


In [0]:
docs=[]
directory = os.fsencode('20news-bydate-test') # Test Data File with 10% data
output_train=[]
d = [0]*20
p = -1
j = os.listdir(directory)
j.sort()
for file in j:
    print(file)
    p+=1
    d[p]=1
    for first in os.listdir(directory+b'/'+file):
        k = directory + b'/' + file + b'/' + first
        with open(k.decode('utf-8'),'r',errors='ignore') as file2:
            output_train.append(d.copy())
            docs.append(file2.read())
    d[p]=0

# Creates a list of docs. Each item is a list of all words of the doc.
m_test=[]
for i in docs:
    i = i.replace('\n',' ')
    m_test.append(text.text_to_word_sequence(i))
test_data1 = m_test

b'alt.atheism'
b'comp.graphics'
b'comp.os.ms-windows.misc'
b'comp.sys.ibm.pc.hardware'
b'comp.sys.mac.hardware'
b'comp.windows.x'
b'misc.forsale'
b'rec.autos'
b'rec.motorcycles'
b'rec.sport.baseball'
b'rec.sport.hockey'
b'sci.crypt'
b'sci.electronics'
b'sci.med'
b'sci.space'
b'soc.religion.christian'
b'talk.politics.guns'
b'talk.politics.mideast'
b'talk.politics.misc'
b'talk.religion.misc'


# WORD2VEC

In [0]:
# Python program to generate word vectors using Word2Vec 
warnings.filterwarnings(action = 'ignore') 

m1 = train_data1 + test_data1  
model = Word2Vec(m1, min_count = 5, size = 200, sg = 1,iter=10)  # Word2Vec Model

In [0]:
len(model.wv.vocab)

43494

In [0]:
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
import gensim 
from gensim.models import Word2Vec 
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts, get_tmpfile

path = get_tmpfile("/content/drive/My Drive/20news-bydate/Word2Vec_14-11-19/wordvectors1.kv")
model.wv.save(path)

'''
# To save the model
path = get_tmpfile("/content/drive/My Drive/20news-bydate/Word2Vec_14-11-19/wordvectors1.model")
model.save(path)

# To load a word2vec model
model = gensim.models.Word2Vec.load('word2vec.model')

# Loading Model KeyedVectors
path = get_tmpfile("/content/drive/My Drive/20news-bydate/Word2Vec(43494)/wordvectors1.kv")
wv = KeyedVectors.load(path, mmap='r')
'''

In [0]:
model.wv.most_similar("university",topn=6)

[('univ', 0.6162004470825195),
 ('nebraska', 0.6018569469451904),
 ('univeristy', 0.5876556038856506),
 ('madeline', 0.5795857906341553),
 ('zilfi', 0.5721766948699951),
 ('wyoming', 0.5688989162445068)]

## Filtering data and creating final data files

In [0]:
# Training/Testing data cleaning to filter out words not in vocabolary but present in the document
train_data=[]
for i in range(len(train_data1)):
    l = filter(lambda x: x in model.wv.vocab,train_data1[i])
    k = []
    for i in l:
        k.append(i)
    train_data.append(k.copy())

test_data=[]
for i in range(len(test_data1)):
    l = filter(lambda x: x in model.wv.vocab,test_data1[i])
    k = []
    for i in l:
        k.append(i)
    test_data.append(k.copy())

In [0]:
# Creating final pickel files
# To save an object in a file

with open("train_data.p", "wb") as fp:   #Pickling
     pickle.dump(train_data, fp)
with open("test_data.p", "wb") as fp:   #Pickling
     pickle.dump(test_data, fp)